In [1]:
# analysis
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# visualization
import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.figure_factory as ff

# preprocessingn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# metrics
from sklearn.metrics import roc_auc_score

# model selection
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

import warnings

warnings.filterwarnings('ignore')

C:\Users\user\anaconda3\envs\ml\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train = pd.read_csv('../../../../../data/pg_epi4/train.csv').drop('id',axis=1)
test = pd.read_csv('../../../../../data/pg_epi4/test.csv')
origin = pd.read_csv('../../../../../data/pg_epi4/creditcard.csv')
train = pd.concat([train,origin],axis=0).drop_duplicates()

In [3]:
train = train.reset_index(drop=True)
train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,...,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,...,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,...,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,...,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,...,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502821,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
502822,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
502823,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
502824,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
train['hour'] = train['Time'] % (24 * 3600) // 3600
train['day'] = (train['Time'] // (24 * 3600))
train = train[(train['Time'] < 120580.0) & (train['hour'] >= 9) & (train['day'] == 0)]
train = train.reset_index(drop=True)

test['hour'] = test['Time'] % (24 * 3600) // 3600
test['day'] = (test['Time'] // (24 * 3600))

target = 'Class'
features = [c for c in train.columns if c not in ['id','Time','day', target]]

In [ ]:
data = train

from pycaret.classification import *
s = setup(data, target = 'Class')

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:33:53
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
Time,Numeric
V1,Numeric
V2,Numeric
V3,Numeric
V4,Numeric
V5,Numeric
V6,Numeric
V7,Numeric
V8,Numeric
V9,Numeric


In [ ]:
best = compare_models()

In [ ]:
# target = 'Class'
# features = ['V1', 'V2', 'V3', 'V4', 'V6', 'V7', 'V8', 'V9', 'V10',
#        'V11', 'V12', 'V13', 'V14', 'V16', 'V17', 'V18', 'V19',
#        'V21', 'V22', 'V24', 'V25', 'V26', 'V27', 'Amount']

# features = [ 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
#        'V11', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
#        'V21', 'V22', 'V24', 'V26', 'V27', 'V28', 'Amount']

# basic test

In [ ]:
import optuna
from optuna.samplers import TPESampler
def rf_objective(trial):
    
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 4, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        }

    n=10
    cv = StratifiedKFold(n,shuffle=True, random_state=42)
    all_scores = []
    for i,(train_idx,val_idx) in enumerate(cv.split(train[features],train[target])):
        X_train, y_train = train.loc[train_idx, features],train.loc[train_idx, target]
        X_val, y_val = train.loc[val_idx, features],train.loc[val_idx, target]

        model = XGBClassifier(**params)
        model.fit(X_train,
                  y_train,
                  eval_set = [(X_val,y_val)],
                  early_stopping_rounds=50,
                  verbose=500)

        y_pred = model.predict_proba(X_val)[:,1]
        score = roc_auc_score(y_val,y_pred)
        all_scores.append(score)

    return np.mean(all_scores)

In [ ]:
# study = optuna.create_study(direction='maximize', sampler = TPESampler())
# study.optimize(func=rf_objective, n_trials=50)
# study.best_params

In [ ]:
# # using_model = [KNeighborsClassifier,
# #                RandomForestClassifier,
# #                ExtraTreesClassifier,
# #                GradientBoostingClassifier,
# #                DecisionTreeClassifier]


# kf = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

# # tar_models = []
# # tar_val_scores = []

# include_orig = True
# # for classifier in using_model:
# models = []
# val_scores = []
# for i, (train_index, val_index) in tqdm(enumerate(kf.split(train, train[target]))):

#     X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
#     y_train, y_val = train[target][train_index], train[target][val_index]

# #     if include_orig:
# #         X_train = X_train.append(original[features], ignore_index = True)
# #         y_train = y_train.append(original[target], ignore_index = True)

#     model = RandomForestClassifier(**study.best_params)

#     model.fit(X_train, y_train)

#     models.append(model)
#     pred = model.predict_proba(X_val)[:,1]
#     score = roc_auc_score(y_val, pred)
#     val_scores.append(score)

#     print(f'AUC on fold {i}: {score:.4f}')

# # tar_models.append(models)
# # tar_val_scores.append(val_scores)

# print(f'mean AUC across all folds: {np.mean(val_scores):.4f}')

In [ ]:
# val_scores

In [ ]:
# plt.figure(figsize = (10, 12))
# cat_imp = np.zeros(len(features))
# for model in models[:10]:
#     cat_imp += model.feature_importances_
    
# plt.barh([features[i] for i in np.argsort(cat_imp)], sorted(cat_imp), 
#          color = "orange", edgecolor = "#000000")

# plt.title("rf")
# plt.show()

In [ ]:
# rf_preds = []

# for model in models[:10]:
#     preds = model.predict_proba(test[features])[:,1]
#     rf_preds.append(preds)

In [ ]:
# rf_preds = np.stack(rf_preds).mean(0)
# rf_preds # cv 10개 모델의 평균

In [ ]:
# rf_df = pd.DataFrame(data = {'id': test.id, 'Class': rf_preds})

In [ ]:
# submission = rf_df
# submission.Class.plot()
# submission.to_csv('submission.csv', index=False)